In [15]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
from dotenv import load_dotenv
load_dotenv()
import requests
BLS_API_KEY = os.getenv("BLS_API_KEY")

In [32]:
def monthly_macro_data_dashboard(start_year='2020', end_year='2025'):
    
    series_dict = {
        'CPI': 'CUSR0000SA0',
        'NFP': 'CES0000000001',
        'Unemployment Rate': 'LNS14000000',
        'JOLTS': 'JTS000000000000000JOL',
        'PPI': 'WPUFD4',
        'Avg Hourly Earnings': 'CES0500000003'
    }

    url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    headers = {'Content-type': 'application/json'}
    payload = {
        "seriesid": list(series_dict.values()),
        "startyear": start_year,
        "endyear": end_year,
        "registrationkey": BLS_API_KEY
    }

    response = requests.post(url, json=payload, headers=headers)
    results = response.json()['Results']['series']

    data_frames = {}

    for series in results:
        series_id = series['seriesID']
        key = [k for k, v in series_dict.items() if v == series_id][0]
        df = pd.DataFrame(series['data'])
        df = df[df['period'].str.startswith('M')]
        df['date'] = pd.to_datetime(df['year'] + '-' + df['period'].str[1:] + '-01')
        df[key] = pd.to_numeric(df['value'])
        df = df[['date', key]].sort_values('date').reset_index(drop=True)

        if key in ['CPI', 'PPI', 'Avg Hourly Earnings']:
            df[f'{key} MoM %'] = df[key].pct_change() * 100
            df[f'{key} YoY %'] = df[key].pct_change(12) * 100

        if key == 'NFP':
            df['NFP MoM Change (k)'] = df['NFP'].diff()

        data_frames[key] = df

    df_merged = data_frames['CPI']
    for key in [k for k in data_frames if k != 'CPI']:
        df_merged = df_merged.merge(data_frames[key], on='date', how='outer')

    df_merged = df_merged.sort_values('date').reset_index(drop=True)

    macro_dashboard = make_subplots(
        rows=3, cols=2, 
        subplot_titles=[
            'SAdj CPI: MoM and YoY (%)',
            'Unemployment Rate: MoM and YoY (%)',
            'NFP (Thousands)',
            'JOLTS',
            'PPI: MoM and YoY (%)',
            'Avg Hourly Earnings'
        ]
    )

    # CPI
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['CPI MoM %'], name='CPI MoM %'), row=1, col=1)
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['CPI YoY %'], name='CPI YoY %'), row=1, col=1)

    # Unemployment Rate
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['Unemployment Rate'], name='Unemployment Rate'), row=1, col=2)
    
    # NFP MoM Change
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['NFP MoM Change (k)'], name='NFP MoM Change (k)'), row=2, col=1)
    
    # JOLTS
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['JOLTS'], name='JOLTS'), row=2, col=2)

    # PPI
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['PPI MoM %'], name='PPI MoM %'), row=3, col=1)
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['PPI YoY %'], name='PPI YoY %'), row=3, col=1)

    # Avg Hourly Earnings Growth
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['Avg Hourly Earnings MoM %'], name='Avg Hourly Earnings MoM %'), row=3, col=2)
    macro_dashboard.add_trace(go.Scatter(x=df_merged['date'], y=df_merged['Avg Hourly Earnings YoY %'], name='Avg Hourly Earnings YoY %'), row=3, col=2)

    macro_dashboard.update_layout(
        height=900,
        # width=800,
        title_text="Macro Dashboard (CPI, NFP, Unemployment Rate, JOLTS, PPI, Earnings)",
        template="plotly_dark",
        showlegend=False
    )

    macro_dashboard.show()

    return df_merged

In [33]:
merged_macro_data = monthly_macro_data_dashboard(start_year=2021)